# Experimentos de optimización: $f(x,y)=y^2 + \log(1+x^2)$

**Autor:** Alina María de la Noval Armenteros  
**Grupo:** C-311  


**Objetivo:** Aplicar los métodos Descenso por Gradiente con Armijo y BFGS (Cuasi-Newton) a la función $f(x,y)=y^2 + \log(1+x^2)$, comparar su comportamiento en términos de convergencia, número de iteraciones, tiempo y sensibilidad al punto inicial, y documentar conclusiones técnicas.

**Resumen breve:** Esta función combina un término cuadrático en $y$ con un término logarítmico en $x$; la diferencia en curvatura entre las direcciones $x$ y $y$ la hace adecuada para comparar la eficacia de métodos de primer y segundo orden.

## Descripción teórica del problema

Función objetivo: $$f(x,y)=y^2 + \log(1+x^2).$$

- Dominio: definida para todo $(x,y)\in\mathbb{R}^2$ ya que $1+x^2>0$ para todo $x$.
- Descomposición: la función se puede escribir como suma de funciones univariadas, $f(x,y)=g(x)+h(y)$ con $g(x)=\log(1+x^2)$ y $h(y)=y^2$. Esto facilita el análisis teórico y numérico.
- Regularidad: ambas componentes son $C^{\infty}$ en el dominio, por lo que $f\in C^{\infty}(\mathbb{R}^2)$; es válido usar métodos que requieran gradiente y aproximaciones de la Hessiana.

**Gradiente:**
$$\nabla f(x,y)=\begin{pmatrix}\dfrac{2x}{1+x^2}\\2y\end{pmatrix}.$$

**Hessiano:**
$$\nabla^2 f(x,y)=\begin{pmatrix}\dfrac{2(1-x^2)}{(1+x^2)^2} & 0\\0 & 2\end{pmatrix}.$$

- Observación sobre convexidad: la componente $(2(1-x^2)/(1+x^2)^2)$ cambia de signo: es positiva para $|x|<1$, cero en $|x|=1$ y negativa para $|x|>1$. Por tanto, el Hessiano no es globalmente semidefinido positivo y la función no es convexa globalmente; sin embargo, localmente alrededor del origen muestra comportamiento convexo.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from optimizacion import gradient_descent_armijo, bfgs
from optimizacion.line_search import backtracking_armijo
from optimizacion.util_json import run_and_save_experiments, load_experiments_from_json
from optimizacion.graficos import plot_convergence, plot_final_vs_iters, plot_trajectory_2d
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(0)

In [ ]:
import numpy as np

def f(x):
    """Función objetivo. `x` es array-like [x, y].
    Devuelve escalar f(x).
    """
    x = np.asarray(x)
    xv = float(x[0])
    yv = float(x[1])
    return yv**2 + np.log(1.0 + xv**2)

def grad(x):
    """Gradiente de f: devuelve array [df/dx, df/dy].
    """
    x = np.asarray(x)
    xv = float(x[0])
    yv = float(x[1])
    dfdx = (2.0 * xv) / (1.0 + xv**2)
    dfdy = 2.0 * yv
    return np.array([dfdx, dfdy])



In [ ]:
algorithms = ['gd', 'bfgs']  
# Puntos actualizados para obtener resultados más diferenciados en las gráficas
points = [np.array([0.05,0.02]), np.array([0.9,0.0]), np.array([2.5,0.0]), np.array([0.0,2.5]), np.array([-3.0,1.5]), np.array([5.0,-2.0])]

tolerance = 1e-6
maxiter = 500

run_configs = []
for alg in algorithms:
    for x0 in points:
        cfg = {
            'algorithm': alg,
            'x0': x0,
            'tolerance': tolerance,
            'line_search': backtracking_armijo if alg == 'gd' else None,
            'maxiter': maxiter
        }
        run_configs.append(cfg)

In [ ]:
# Mapear nombres a funciones (coincide con 'algorithm' en run_configs)
algorithm_fn_map = {
    'gd': gradient_descent_armijo,
    'bfgs': bfgs
}

# Ejecutar y guardar resultados (guardará en data/resultados/experimentos.json)
output_file = 'data/resultados/experimentos.json'
experiment_data = run_and_save_experiments(run_configs, algorithm_fn_map, filename=output_file, f=f, grad=grad)


In [ ]:
# Cargar resultados desde JSON para análisis/plotting
data = load_experiments_from_json('data/resultados/experimentos.json')
experiments = data.get('experiments', [])

In [ ]:
# Graficación: convergencia, comparación y trayectorias individuales
import os
os.makedirs('data/resultados', exist_ok=True)
plot_convergence(experiments, filename='data/resultados/convergencia.png', group_by='algorithm', smooth_window=5, decimate=2, legend_outside=True, legend_fontsize='x-small', compact_labels=True)
plot_final_vs_iters(experiments, filename='data/resultados/final_vs_iters.png')
if experiments:
    exp0 = experiments[0]
    plot_trajectory_2d(exp0, filename='data/resultados/trayectoria_exp1.png')

## Análisis de resultados y conclusiones

Interpreta las gráficas y la tabla de resultados atendiendo a los siguientes puntos técnicos:

1. Convergencia: compara número de iteraciones y tiempo (CPU) entre GD-Armijo y BFGS. Indica si alguna corrida no alcanzó la tolerancia antes de `maxiter` y reporta el estado final.

2. Calidad de la solución: compara $f_{final}$ y $\|\nabla f\|_{final}$ entre algoritmos y puntos iniciales; identifica si hay soluciones con valores significativamente distintos.

3. Sensibilidad al punto inicial: analiza cómo cambian resultados y número de iteraciones con diferentes $x_0$; comenta si hay regiones problemáticas (p. ej. |x|>1 con curvatura negativa en x).

4. Efecto de la curvatura: observa comportamientos típicos (pasos pequeños en $x$, oscilaciones) ligados a la variación de la componente Hessiana en $x$.

5. Recomendaciones: estima cuándo usar GD-Armijo (simples, bajo coste por iteración) y cuándo BFGS (menos iteraciones, mejor en problemas con anisotropía de curvatura). Considera L-BFGS para dimensiones mayores o Newton si la Hessiana es accesible.

Sugerencia para el informe final:
- Incluir una tabla resumen con columnas: algoritmo, x0, tol, iters, tiempo, f_final, grad_norm, status.
- Adjuntar las figuras guardadas en `data/resultados/` y comentar patrones relevantes.
- Concluir con una recomendación clara: cuándo preferir cada método y por qué.